<a href="https://colab.research.google.com/github/kevinracso/01Tarea/blob/master/Copia_de_Copia_de_Preprocesamiento_y_Red_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Permite importar archivos desde Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importar librerías

In [0]:
import numpy as np
import pandas as pd
import matplotlib
#%matplotlib inline
import matplotlib.pyplot as plt

import matplotlib.cm as cm
from skimage import io, img_as_float
import scipy.interpolate as itp

import random

In [0]:
from __future__ import print_function
from time import time

from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

from warnings import simplefilter

simplefilter(action='ignore', category=FutureWarning) # Comando para ignorar los warnings


import keras  # Importando la libreria keras
from keras.models import Sequential  
from keras.layers import Dense, Dropout, Flatten # Comandos de keras que importan las capas dense, dropout y flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D # Comandos de keras que importan las capas convolucionales y maxpooling2D
from keras import backend as K
from keras.layers.advanced_activations import LeakyReLU

Using TensorFlow backend.


## Cargar base de datos 

In [0]:
# Cargando base de datos Alerce.
data = pd.read_pickle('/content/drive/My Drive/Supernova/ALERCE_stamps.pkl') # Base de datos original
#data = pd.read_pickle('/content/drive/My Drive/Supernova/ALERCE_stamps_1k.pkl') # Base de datos reducida

In [0]:
imgs = data['images'] # Lista de 31727 imagenes
labels = data['labels'] # Lista de 31727 labels

img_0 = imgs[0] # Primera imagen de imgs, ndarray 63x63, 3 canales
label_0 = labels[0] # Primer label, entero

In [0]:
np.shape(imgs)

(31727,)

## Desechar imágenes no cuadradas

In [0]:
shap_def = (63, 63, 3) # Estructura correcta de imagenes, cuadradas, 3 canales

img_sq = [] # Inicializar lista vacia para almacenar imagenes que tienen
            # estructura correcta
label_sq = [] # Inicializar lista vacia para almacenar labels de imagenes que
              # tienen estructura correcta
    
for j in list(range(len(imgs))): # recorrer imagenes
    #print('Revisando si la imagen '+str(j)+' es cuadrada')
    if np.shape(imgs[j]) == shap_def: # si tiene estructura correcta, agregar
        
        img_sq.append(imgs[j])
        label_sq.append(labels[j])

## Normalizar

In [0]:
img_sq = np.asarray(img_sq)
label_sq = np.asarray(label_sq)

# Funcion auxiliar que normaliza UNA matriz
def Norm_Data_matrix(matrix):
    max_data = np.nanmax(matrix)
    min_data = np.nanmin(matrix)
    Norm_matrix = (matrix-min_data)/(max_data-min_data)
    return Norm_matrix

# Normalizacion de las imagenes
img_norm = []
for i in range(len(img_sq)):
    muestra = img_sq[i]
    #print('Normalizando muestra ' + str(i))
    muestra[:, :, 0] = Norm_Data_matrix(muestra[:, :, 0])
    muestra[:, :, 1] = Norm_Data_matrix(muestra[:, :, 1])
    muestra[:, :, 2] = Norm_Data_matrix(muestra[:, :, 2])
    img_norm.append(muestra)


In [0]:
img_norm = np.array(img_norm) # Convertir a arreglo numpy

nan_index = np.argwhere(np.isnan(img_norm))
#print('Indices de nan en arreglo normalizado', nan_index)

img_no_nan = np.nan_to_num(img_norm) # Convertir nan a cero

nonan_index = np.argwhere(np.isnan(img_no_nan))
#print('Indices de nan en arreglo al eliminar nan', nonan_index)


In [0]:
np.shape(img_no_nan) # Dimensiones de la base corregida

(31228, 63, 63, 3)

## Balance de Clases, definir conjuntos de train, val, y test

In [0]:
"Objetos a tratar"

" Clases = {0: 'AGN', 1:'SN', 2:'VS', 3:'asteroid', 4:'bogus'}" 
"AGN: Active Galactic Nuclei"
"SN: Supernova"
"VS: Variable Star"
"bogus: Artifacts"

img_no_nan # imagenes, np array
label_sq # labels, np array

unique, counts = np.unique(label_sq, return_counts=True)
class_counter = dict(zip(unique, counts)) # muestra cuantas veces se repite cada clase
print('Numero de muestras por clase:', class_counter)

#test = np.concatenate((img_no_nan, label_sq), axis=0)
#test_1 = np.concatenate((img_no_nan, label_sq), axis=1)

s_index = np.arange(label_sq.shape[0]) # indexar arreglos

np.random.shuffle(s_index) # revolver

# label_sq.shape[0], label_sq.shape

img_no_nan = img_no_nan[s_index] # reemplazar arreglos originales por arreglos revueltos
label_sq = label_sq[s_index]

"Encontrar ubicaciones de todos los tipos de objeto"

agn_ind = np.squeeze(np.argwhere(label_sq == 0))
sn_ind = np.squeeze(np.argwhere(label_sq == 1))
vs_ind = np.squeeze(np.argwhere(label_sq == 2))
ast_ind = np.squeeze(np.argwhere(label_sq == 3))
bog_ind = np.squeeze(np.argwhere(label_sq == 4))

"Una lista para cada tipo de objeto"

img_AGN = img_no_nan[agn_ind]
img_SN = img_no_nan[sn_ind]
img_VS = img_no_nan[vs_ind]
img_AST = img_no_nan[ast_ind]
img_bog = img_no_nan[bog_ind]




Numero de muestras por clase: {0: 8721, 1: 987, 2: 9772, 3: 9797, 4: 1951}
[0 0 0 ... 0 0 0]
[ True  True  True ...  True  True  True]


In [0]:
# Separación de la base de datos en un conjunto de entrenamiento y validación.
train_size = int(len(img_no_nan)*.75) # Definiendo el tamaño del conjunto de entrenamiento con los requisitos solicitados 
train_total = int(len(img_no_nan)) # Tamaño total del conjunto original.
x_train = img_no_nan[0:train_size,:] # Nuevo conjunto horizontal de entrenamiento.
x_val = img_no_nan[train_size:train_total,:] # Conjunto de validacion horizontal. (imagenes)
y_train = y_data[:train_size] # Nuevo conjunto vertical de entrenamiento. (clases)
y_val = y_data[train_size:] # Conjunto de validacion vertical (clases)

input_shape = (img_rows, img_cols, 3)

# Red Neuronal

## Preliminares

In [0]:
# Para poder procesar las clases es necesario pasarlas a one hot encoding

# Funcion para tener las clases como one hot encoding
def one_hot(a, num_classes):
    return np.squeeze(np.eye(num_classes)[a.reshape(-1)])



In [0]:
onehot_labels = one_hot(label_sq, 5)

In [0]:
batch_size = 128 # Definiendo el tamaño de los batches
num_classes = 5 # Numero de clases del conjunto
epochs = 12 # Cantidad de epocas
#pool_value = 0.5

In [0]:
# input image dimensions
img_rows, img_cols = 63, 63  # Definiendo las filas y columnas de las imagenes
y_data = onehot_labels

In [0]:
np.shape(img_no_nan)

(31228, 63, 63, 3)

## Al parecer este bloque no hace nada a las variables x_train y similares

In [0]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_train /= 255
x_val /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_val.shape[0], 'test samples')

x_train shape: (23421, 63, 63, 3)
23421 train samples
7807 test samples


## Configurar Red!

In [0]:
""" PROGRAMACIÓN DE LA RED NEURONAL CONVOLUCIONAL!!
    En los siguientes bloques se configura una CNN usando la estructura entregada en el paper.
"""


"Convierten a one hot encoding"
y_train_1 = keras.utils.to_categorical(y_train, num_classes) # Categorización de las muestras del conjunto vertical de entrenamiento.
y_val_1 = keras.utils.to_categorical(y_val, num_classes) # Categorización de las muestras del conjunto vertical de prueba.
"Convierten a one hot encoding"


time_i = time()
model = Sequential() # Inicio de la configuracion del clasificador , a partir de este punto se le añadiran las capas que conformaran la CNN
model.add(Conv2D(32, kernel_size=(4, 4), # Añadiendo una capa convolucional al modelo con un kernel  de 3x3 
                 activation = None, # Con un funcion de activacion relu  y las dimensiones configuradas en el bloque anterior
                 input_shape=input_shape)) 
model.add(Conv2D(32, (3, 3), activation = None)) # Añadiendo otra capa convolucional  con una funcion de activacion relu
model.add(MaxPooling2D(pool_size=(2, 2),strides = 2)) # Utilizando el método maxpooling en esta red.
model.add(Conv2D(64, (3, 3), activation = None))
model.add(Conv2D(64, (3, 3), activation = None))
model.add(Conv2D(64, (3, 3), activation = None))
model.add(MaxPooling2D(pool_size=(2, 2), strides = 2))
#model.add(Dropout(0.5)) # Añadiendo Dropout a la red
model.add(Flatten()) # Este comando remueve todas las dimensiones de un tensor exceptuano una.
#model.add(Dense(128, activation=LeakyReLU)) # En esta capa se implementa la funcion de activacion relu a la capa anterior de la la red neuronal. 
model.add(Dense(64)) # 64 salidas
model.add(LeakyReLU()) # LeakyReLU
model.add(Dropout(0.5)) # Se agrega la tecnica de Dropout para abordar el overfitting.
model.add(Dense(64))
model.add(LeakyReLU()) # LeakyReLU
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax')) # Se aplica a la red siendo esta vez usando la funcion softmax.

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy']) # Con este coomando se configura el modelo para el entrenamiento.







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




## Entrenamiento

In [0]:
# Entrenamiento del modelo

model_train = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_val, y_val)) # Comando para entrenar el modelo.
model_train
score = model.evaluate(x_val, y_val, verbose=0) # Con este comando se obtiene el valor de perdida y de las metricas en el modo de prueba.
time_f = time()
time_execution = time_f - time_i # Con este comando se obtiene el valor de perdida y de las metricas en el modo de prueba.
print('Valid loss:', score[0]) # Se imprime la perdida para la prueba realizada.
print('Valid accuracy:', score[1]) # Se imprimre el accuracy para la prueba realizada.
print('Tiempo de entrenamiento ' + str(time_execution))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 23421 samples, validate on 7807 samples
Epoch 1/12
 7296/23421 [========>.....................] - ETA: 24s - loss: 1.4488 - acc: 0.3076

KeyboardInterrupt: ignored

In [0]:
import numpy as np

x = np.array([0, 3, 1, 2, 4, 1, 0, 4])
# x = np.reshape(x, (4, 25))
print(x)


ind = np.squeeze(np.argwhere(x%7 == 0))
print(ind)

#y = np.squeeze(x[ind])
y = x[ind]
print(y)

z = keras.utils.to_categorical(x, 5)

print(z)

[0 3 1 2 4 1 0 4]
[0 6]
[0 0]
[[1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]]
